In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
mm = MinMaxScaler()
ss = StandardScaler()



In [4]:
#등락으로 분류를 시켰기 때문에 등락 퍼센트는 제거해 주었습니다.다른 변수와 상관성이 너무 높은 속성들은 제거처리 해주었습니다.
df=pd.read_csv(r'C:\DRworks\shynix_short.csv',encoding='cp949', index_col=0)
df=df.drop('등락분류',axis=1)
df=df.drop('등락률',axis=1)
df=df.drop('대비',axis=1)
df=df.drop('등락지표',axis=1)
df=df.drop('등락',axis=1)
df=df.drop('시가',axis=1)
df=df.drop('고가',axis=1)
df=df.drop('저가',axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2670 entries, 2010-07-19 to 2021-05-14
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   삼성등락       2670 non-null   int64  
 1   종가         2670 non-null   float64
 2   거래량        2670 non-null   int64  
 3   거래대금       2670 non-null   float64
 4   시가총액       2670 non-null   float64
 5   상장주식수      2670 non-null   int64  
 6   환율         2670 non-null   float64
 7   유가종가       2670 non-null   float64
 8   유가오픈       2670 non-null   float64
 9   유가고가       2670 non-null   float64
 10  유가저가       2670 non-null   float64
 11  유가변동률      2670 non-null   float64
 12  한국은행기준금리   2670 non-null   float64
 13  연준기준금리     2670 non-null   float64
 14  한미기준금리차이   2670 non-null   float64
 15  반도체수출금액지수  2661 non-null   float64
 16  삼성종가       2670 non-null   float64
 17  SOX종가      2670 non-null   float64
 18  SOX변동률     2670 non-null   float64
 19  BTC종가      2670 non-null   float64
 20

In [5]:
#xbbost를 위해 x y 로 데이터 분리 후 dmatrix로 변환 (y=등락정도)
X, y = df.iloc[:,1:],df.iloc[:,0]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
#시기로 분할할 때 데이터
#split = "2012-03-19"
#train = df[:split]
#test= df[split:]

#y_train = train.loc[:,'등락.분류']
#X_train = train.drop('등락.분류', axis=1)
#y_test = test.loc[:,'등락.분류']
#X_test = test.drop('등락.분류', axis=1)


In [6]:

#시기로 분할할 때 데이터
#split = "2012-03-19"
#train = df[:split]
#test= df[split:]

#y_train = train.loc[:,'등락.분류']
#X_train = train.drop('등락.분류', axis=1)
#y_test = test.loc[:,'등락.분류']
#X_test = test.drop('등락.분류', axis=1)

# XGBoost 분류기 생성
xgb_clf = xgb.XGBClassifier()

#초모수 격자생성
xgb_param_grid = {   
                    
                    'gamma':[0,1,2], 
                    'max_depth':[6,7,8,9], 
                    'min_child_weight':[3,4,5], 
                    'n_estimators':[50,100,150,200], 
                    'nthread':[3,4,5,6],                    
                    'random_state':[50,100,150,200] 
                    }
# GridSearchCV 객체 생성
hr_grid = GridSearchCV(estimator=xgb_clf,
                       param_grid=xgb_param_grid,
                       scoring='roc_auc',
                       n_jobs=8,
                       cv=5,
                       refit=True, 
                       return_train_score=True)
hr_grid.fit(X_train, y_train)
#model=XGBClassifier(booster='gbtree', 
                   # colsample_bylevel=0.9, 
                   # colsample_bytree=0.8, 
                   # gamma=0, 
                   # max_depth=8, 
                   # min_child_weight=4, 
                   # n_estimators=100, 
                   # nthread=4, 
                   # objective='multi:softprob', 
                   # random_state=150, 
                   # silent= True)

#model.fit(X_train, y_train,verbose=False)

C:\phythontemp\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:03:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [7]:
## 최고성능
best_score = hr_grid.best_score_
# 최고성능을 내는 행을 찾아냄
best_row = hr_grid.best_index_

# 최적 초모수: max_depth, subsample
best_max_depth     = hr_grid.best_params_["max_depth"]
best_max_gamma = hr_grid.best_params_["gamma"]
best_max_min_child_weight= hr_grid.best_params_["min_child_weight"]
best_max_n_estimators= hr_grid.best_params_["n_estimators"]
best_max_nthread= hr_grid.best_params_["nthread"]
best_max_random_state= hr_grid.best_params_["random_state"]


nl = '\n'
print(f 'AUC:  \t {best_score:.6f}{nl}\
         index:           \t {best_row}{nl}\
         max_depth:      \t {max_depth}{nl}\
         gamma :    \t{gamma}{nl}\
         min_child_weight: \t{min_child_weight}{nl}\
         n_estimators:  \t{n_estimators}{nl}\
         nthread :   \t{nthread}{nl}\
         random_state:  \t{random_state}'
     )

SyntaxError: invalid syntax (<ipython-input-7-3093fa8f89ba>, line 23)

In [8]:
hr_grid_df = pd.DataFrame(hr_grid.cv_results_)
hr_grid_df.loc[:,['mean_test_score', "params"]]
hr_grid_df.sort_values(by=['mean_test_score'],ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_max_depth,param_min_child_weight,param_n_estimators,param_nthread,param_random_state,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1857,0.617264,0.018154,0.005205,7.491184e-04,2,7,5,50,3,100,...,0.820108,0.02742,1,0.991415,0.9981,0.995648,0.997696,0.995376,0.995647,0.002375
1889,1.340723,0.068770,0.004805,4.001379e-04,2,7,5,150,3,100,...,0.820108,0.02742,1,0.991415,0.9981,0.995648,0.997696,0.995376,0.995647,0.002375
1891,1.567031,0.099832,0.005205,4.003526e-04,2,7,5,150,3,200,...,0.820108,0.02742,1,0.991415,0.9981,0.995648,0.997696,0.995376,0.995647,0.002375
1892,1.718773,0.066742,0.005202,4.019165e-04,2,7,5,150,4,50,...,0.820108,0.02742,1,0.991415,0.9981,0.995648,0.997696,0.995376,0.995647,0.002375
1893,1.806155,0.068590,0.005205,7.490547e-04,2,7,5,150,4,100,...,0.820108,0.02742,1,0.991415,0.9981,0.995648,0.997696,0.995376,0.995647,0.002375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,1.724677,0.005272,0.005205,4.004002e-04,0,6,3,200,4,100,...,0.806844,0.02810,2289,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,0.000000
52,1.675430,0.023584,0.005005,9.536743e-08,0,6,3,200,4,50,...,0.806844,0.02810,2289,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,0.000000
51,1.667124,0.088899,0.005906,4.909921e-04,0,6,3,200,3,200,...,0.806844,0.02810,2289,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,0.000000
50,1.458933,0.388948,0.005806,4.003525e-04,0,6,3,200,3,150,...,0.806844,0.02810,2289,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,0.000000


In [11]:
#변수 중요도 그래프
plt.rc('font',family='Malgun Gothic')
plt.rcParams["figure.figsize"] = (10,7)
plot_importance(hr_grid_df, height=0.9)

ValueError: tree must be Booster, XGBModel or dict instance

In [ ]:
#모델 예측
#y_pred = model.predict(X_test)
#y_true = y_test

#acc = accuracy_score(y_true, y_pred)
#con_mat = confusion_matrix(y_true, y_pred)
#con_mat
#report = classification_report(y_true, y_pred)
#print(report)